# Export from Labelbox to Voxel51
After you have finished labeling data in LabelBox, this notebook lets you import the labels back into a Voxel51 Dataset.
In the Labelbox web UI, export the project and download the JSON file. 

In [ ]:
labelboxExportJson =  "/tf/notebooks/export-2021-02-01T01-34-34.538Z.json" # Download the exported JSON and update this
dataset_name = "test-dataset" # The name of the V51 Dataset to use
labelbox_id_field = "labelbox_id" # V51 Sample field where the corresponding Labelbox ID was save when it was uploaded to Labelbox

## Labelbox Export

In [ ]:
# Imports and configuration 
import fiftyone as fo

In [ ]:
# Do the groundwork for importing, setup the dataset
import fiftyone.utils.labelbox as foul
from uuid import uuid4

# expect an error here if the dataset already exists
dataset = fo.load_dataset(dataset_name)

In [ ]:
# Imports the Data from Labelbox into a Voxel51 Dataset
foul.import_from_labelbox(dataset, labelboxExportJson,  labelbox_id_field=labelbox_id_field, download_dir="/tf/media")

### Examine the results

In [ ]:
session = fo.launch_app(dataset, auto=False)

## Post Processing
You may want to do some additional data munging. I added a tag based on whether a plane was labeled or skipped in Labelbox.

In [ ]:
# Add a label & tag that captures if the image was skipped, indicating there was no plane, or accepted, indicating there was a plane
from fiftyone import ViewField as F
label_field = "plane_ground_truth" 

model_view = dataset.exists("model")
for sample in model_view:
    sample[label_field] = fo.Classification(label="plane")
    sample.tags.append("plane")
    sample.save()


skipped_view = dataset.match({"model": {"$exists": False, "$eq": None}})
for sample in skipped_view:
    #print(sample)
    sample[label_field] = fo.Classification(label="noplane")
    sample.tags.append("noPlane")
    sample.save()    